In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load the new dataset
dataset_name = "TuringsSolutions/PFAF750"
dataset = load_dataset(dataset_name)

# Inspect the structure of the dataset
print("Dataset structure and sample data:")
print(dataset)  # Overview of dataset splits
print(dataset['train'][0])  # Sample from the training split

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)FAF Function LLM Training - Sheet1-4.csv:   0%|          | 0.00/510k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/757 [00:00<?, ? examples/s]

Dataset structure and sample data:
DatasetDict({
    train: Dataset({
        features: ['Prompt', 'Response'],
        num_rows: 757
    })
})
{'Prompt': 'What is the P-FAF function?', 'Response': 'The Probabilistic Fractal Activation Function (P-FAF):\nEquation:\nP-FAF(x) = ∑(p_i * f_i(x^(1/d_i)))\nWhere:\nx is the input value\np_i are probabilities for each fractal function (sum to 1)\nf_i(x) are different fractal functions (e.g., sierpinski triangle, mandelbrot set)\nd_i are the dimensions of each fractal function\nEssentially, this equation takes an input value (x) and passes it through various fractal functions, each raised to a non-integer power (dimension). The output is a weighted sum of the individual fractal function outputs, where the weights are probabilities.'}


In [ ]:
print(dataset['train'].features)

{'Response': Value(dtype='string', id=None), 'Unnamed: 1': Value(dtype='string', id=None)}


In [ ]:
for example in dataset['train']:
    print(example)
    break  # Print only the first example to confirm its structure

{'Response': 'To update node embeddings in a GNN, consider aggregating information from neighboring nodes, weighted by their importance in the graph structure.', 'Unnamed: 1': None}


In [ ]:
import random
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from difflib import SequenceMatcher
import re

# Load PFAF dataset and initialize model
dataset = load_dataset("TuringsSolutions/PFAF750")  # Corrected with quotes
checkpoint = "HuggingFaceTB/SmolLM2-360M"
device = "cpu"  # Use "cuda" for GPU or "cpu" for CPU
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# For multiple GPUs, install accelerate and use `device_map="auto"`
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

# Assign a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Resize embeddings to accommodate new token

# Optimizer for fine-tuning
optimizer = AdamW(model.parameters(), lr=5e-5)

# State Tracker
state_tracker = {
    "accuracy_score": 0,
    "complexity_level": 1  # Starts with level 1
}

# Text Normalization Function
def normalize_text(text):
    """
    Normalize text by removing extra spaces, punctuation, and converting to lowercase.
    """
    text = re.sub(r'\s+', ' ', text)  # Collapse multiple spaces into one
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.strip().lower()

# Fuzzy Matching Function
def is_similar(response, correct_response, threshold=0.9):
    """
    Check if the model's response is similar to the correct response based on a threshold.
    """
    normalized_response = normalize_text(response)
    normalized_correct_response = normalize_text(correct_response)
    similarity = SequenceMatcher(None, normalized_response, normalized_correct_response).ratio()
    return similarity >= threshold

# Feedback Generator
def feedback_generator(prompt, model_response, correct_response):
    if is_similar(model_response, correct_response):
        return f"Correct! The answer is indeed: {correct_response}."
    else:
        return (f"Incorrect. The correct answer is: {correct_response}. "
                f"Your response was: {model_response.strip()}.")

# Fine-tuning Function
def fine_tune_model(prompt, correct_response):
    model.train()  # Set model to training mode
    # Tokenize inputs and correct response
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True)
    labels = tokenizer(correct_response, return_tensors="pt", truncation=True, padding=True)["input_ids"]

    # Ensure the labels match the input shape
    labels = torch.nn.functional.pad(labels, (0, inputs["input_ids"].size(1) - labels.size(1)), value=tokenizer.pad_token_id)

    # Forward pass with labels for supervised learning
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update model weights
    optimizer.zero_grad()  # Reset gradients
    return loss.item()

# Construct Prompt Function
def construct_prompt(example, default_prompt="Provide a detailed explanation for this response:"):
    """
    Construct a prompt dynamically if the prompt field is missing.
    """
    if 'Prompt' not in example or example['Prompt'] is None:
        return f"{default_prompt}\n{example.get('Response', '')}"
    return example['Prompt']

# Adaptive Boundary Experiment Function
def adaptive_boundary_experiment(dataset, epochs=1, max_length=256):
    results = []
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}\n" + "=" * 50)

        # Process the training split
        data_split = dataset['train']
        for example in data_split:
            # Construct prompt if missing
            prompt = construct_prompt(example)
            correct_response = example.get('Response', None)

            if correct_response is None:
                print(f"Skipping example due to missing response: {example}")
                continue

            print(f"Prompt: {prompt}")
            # Generate reasoning using LLM
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True)
            outputs = model.generate(**inputs, max_length=max_length)
            model_response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
            print(f"Model Response: {model_response}")

            # Generate feedback
            feedback = feedback_generator(prompt, model_response, correct_response)
            print(f"Feedback: {feedback}")

            # Update state tracker and fine-tune if incorrect
            if is_similar(model_response, correct_response):
                state_tracker["accuracy_score"] += 10
            else:
                state_tracker["accuracy_score"] -= 5
                # Fine-tune the model
                loss = fine_tune_model(prompt, correct_response)
                print(f"Fine-tuned on incorrect response. Loss: {loss:.4f}")

            # Store results
            results.append({
                "epoch": epoch + 1,
                "prompt": prompt,
                "model_response": model_response,
                "correct_response": correct_response,
                "feedback": feedback,
                "similarity_score": SequenceMatcher(None, normalize_text(model_response), normalize_text(correct_response)).ratio()
            })
            print("\n" + "-" * 50)

        print("Completed all examples for this epoch.")

    return results

# Run the experiment
epochs = 3  # Number of epochs
results = adaptive_boundary_experiment(dataset, epochs)

# Analyze Results
correct_count = sum(1 for r in results if is_similar(r["model_response"], r["correct_response"]))
print(f"Accuracy: {correct_count}/{len(results)} ({correct_count / len(results) * 100:.2f}%)")


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)FAF Function LLM Training - Sheet1-4.csv:   0%|          | 0.00/510k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/757 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Fine-tuned on incorrect response. Loss: 11.3366

--------------------------------------------------
Prompt: What are inductive datatypes?
Model Response: What are inductive datatypes? of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of  of  of  of of of of of of of of of of  of  of  of  of of of of  of  of  of  of  of  of of of of of of of of of of of of of of of of  of  of of of of of of of of of of of of of  of  of  of of of  of
Feedback: Incorrect. The correct answer is: Inductive datatypes denote elements constructed from primitive 

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Fine-tuned on incorrect response. Loss: 10.4000

--------------------------------------------------
Prompt: Explain pattern matching.


KeyboardInterrupt: 